In [74]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

from keras.layers import Input, Dense
from keras.models import Model
from keras.layers.noise import GaussianNoise
from keras.datasets import mnist
from keras import optimizers

# clasificadores
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score

import time
import pandas as pd
import numpy as np

#variables para los datasets
data_names = ['amazon', 'twitter', 'twitter_3']

#variables para guardar los resultados
tipo = 'sda'

In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print x_train.shape
print x_test.shape

In [ ]:
input_img = Input(shape=(784, ))
encoded = Dense(128, activation='relu')(input_img)
encoded = GaussianNoise(1)(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [ ]:
encoder = Model(input_img, encoded)

In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
               epochs=1,
               batch_size=256,
               shuffle=True,
               validation_data=(x_test,x_test))

In [ ]:
transformed = encoder.predict(x_test[1:2])

# Desde aqui importa

In [31]:
def create_SDA(input_size, layers):
    input_layer = Input(shape=(input_size,))    
    encoded = None
    decoded = None
    
    for layer in layers:
        if encoded is None:
            encoded = Dense(layer, activation='sigmoid')(input_layer)
            #masking noise
            encoded = GaussianNoise(1)(encoded)
        else:
            encoded = Dense(layer, activation='softplus')(encoded)
            
    
    for layer in layers[-2::-1]:
        if decoded is None:
            decoded = Dense(layer, activation='relu')(encoded)
        else:
            decoded = Dense(layer, activation='relu')(decoded)
       
    if len(layers) == 1:
            decoded = Dense(input_size, activation='sigmoid')(encoded)
    else:
        decoded = Dense(input_size, activation='sigmoid')(decoded)
    
    encoder = Model(input_layer, encoded)
    
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='sgd', loss='binary_crossentropy')
    
    return autoencoder, encoder

# Dataset de Amazon

In [38]:
# cargando dataset Amazon
dataset_path = save_path+data_names[0]+'.pkl'
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


In [41]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X)

print "Todos los datos disponibles obtenidos"

Todos los datos disponibles obtenidos


In [40]:
dims = 1000

In [48]:
#autoencoder, encoder = create_SDA(784, [128, 64, 32, 16, 8])
autoencoder, encoder = create_SDA(dims, [500, 300])

In [50]:
X_train = X[:, :dims]

autoencoder.fit(X_train, X_train,
               epochs=15,
               batch_size=256,
               shuffle=True,
               validation_data=(X_train, X_train))

Train on 27677 samples, validate on 27677 samples
Epoch 1/15
27677/27677 [==============================] - 13s - loss: 0.1163 - val_loss: 0.1190
Epoch 2/15
27677/27677 [==============================] - 13s - loss: 0.1064 - val_loss: 0.1092
Epoch 3/15
27677/27677 [==============================] - 13s - loss: 0.0998 - val_loss: 0.1026
Epoch 4/15
27677/27677 [==============================] - 13s - loss: 0.0955 - val_loss: 0.0980
Epoch 5/15
27677/27677 [==============================] - 14s - loss: 0.0924 - val_loss: 0.0947
Epoch 6/15
27677/27677 [==============================] - 15s - loss: 0.0903 - val_loss: 0.0922
Epoch 7/15
27677/27677 [==============================] - 14s - loss: 0.0888 - val_loss: 0.0903
Epoch 8/15
27677/27677 [==============================] - 15s - loss: 0.0876 - val_loss: 0.0889
Epoch 9/15
27677/27677 [==============================] - 15s - loss: 0.0867 - val_loss: 0.0878
Epoch 10/15
27677/27677 [==============================] - 16s - loss: 0.0861 - val_lo

In [65]:
#diccionario para mantener los dominios adaptados
adapted = {}

for domain in domains:
    print "Adaptando %s" % domain
    X_tr = np.asarray(labeled[domain]['X_tr'][:, :dims].todense())
    X_ts = np.asarray(labeled[domain]['X_ts'][:, :dims].todense())
    
    tr_reps = encoder.predict(X_tr[:, :dims])
    ts_reps = encoder.predict(X_ts[:, :dims])
    
    adapted[domain] = {
        'X_tr': tr_reps,
        'X_ts': ts_reps
    }

Adaptando kitchen
Adaptando dvd
Adaptando electronics
Adaptando books


In [71]:
df = pd.DataFrame(columns=['Adaptacion','Dataset','Tarea','Fuente', 'Objetivo','Baseline error', 'Transfer error', 'Transfer loss'])


i = 0
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'][:, :dims].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'][:, :dims].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            #svc = get_best_score(X_tr, y_tr)
            
            svc = SVC(kernel='linear')
            svc = svc.fit(X_tr, y_tr)
            
            b_error = 1-svc.score(X_ts, y_ts)
            
            #transfer error
            #entrenado en dominio src y probado en dominio tgt adaptados
            X_tr_a = adapted[src]['X_tr']
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts_a = adapted[tgt]['X_ts']
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            svc_a = SVC(kernel='linear')
            svc_a = svc_a.fit(X_tr_a, y_tr)
            
            t_error = 1-svc_a.score(X_ts_a, y_ts)
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['SDA','Amazon',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            i+=1

print "Pruebas completadas."

Tarea 1 de 12
0.28  - 0.368
Tarea 2 de 12
0.2875  - 0.360
Tarea 3 de 12
0.3225  - 0.420
Tarea 4 de 12
0.24  - 0.397
Tarea 5 de 12
0.2875  - 0.410
Tarea 6 de 12
0.3225  - 0.360
Tarea 7 de 12
0.24  - 0.380
Tarea 8 de 12
0.28  - 0.365
Tarea 9 de 12
0.3225  - 0.432
Tarea 10 de 12
0.24  - 0.377
Tarea 11 de 12
0.28  - 0.310
Tarea 12 de 12
0.2875  - 0.385
Pruebas completadas.


In [72]:
df

,Adaptacion,Dataset,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,Baseline,Amazon,k->d,kitchen,dvd,28.00,36.75,8.75
1,Baseline,Amazon,k->e,kitchen,electronics,28.75,36.00,7.25
2,Baseline,Amazon,k->b,kitchen,books,32.25,42.00,9.75
3,Baseline,Amazon,d->k,dvd,kitchen,24.00,39.75,15.75
4,Baseline,Amazon,d->e,dvd,electronics,28.75,41.00,12.25
5,Baseline,Amazon,d->b,dvd,books,32.25,36.00,3.75
6,Baseline,Amazon,e->k,electronics,kitchen,24.00,38.00,14.00
7,Baseline,Amazon,e->d,electronics,dvd,28.00,36.50,8.50
8,Baseline,Amazon,e->b,electronics,books,32.25,43.25,11.00
9,Baseline,Amazon,b->k,books,kitchen,24.00,37.75,13.75


In [76]:
df.to_csv(scores_path+tipo+'/amazon.csv', columns=df.columns)